 # Notebook 04 — Calibration & Statistical Validation



 Contains the computationally heavy validation steps:

 - Block bootstrap 95 % confidence intervals on slope, intercept, and RMSE

 - Permutation test (circular shifts + block permutations)

 - Flat-master null model comparison

 - Multi-element screening table per branch



 Workflow:

 1. Full pipeline re-run (auth → align → calibrate)

 2. Block bootstrap CIs for Mg/Sr calibration

 3. Permutation test for Mg/Sr

 4. Flat-master null model comparison

 5. Multi-element screening per branch

 ## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install dtaidistance

import sys

# Edit REPO_PATH if you cloned the repo to a different location inside MyDrive
REPO_PATH = '/content/rhodopipeline'
if REPO_PATH not in sys.path:
    sys.path.insert(0, REPO_PATH)

import rhodopipeline
from rhodopipeline import RhodolithPipeline, CONFIG

pipeline = RhodolithPipeline(CONFIG)
pipeline.authenticate()
pipeline.load_temperature_data()
pipeline.load_curve6_curve7()
pipeline.screen_best_linear_branch()
pipeline.generate_synthetic_master()
pipeline.perform_dtw_alignment(window_days=30)
pipeline.build_composite_and_calibrate()

print('\nPipeline ready for validation.')


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 40.3 MB/s eta 0:00:00
STEP 1: AUTHENTICATION
Mounted at /content/drive
✓ Authenticated.

STEP 2: LOAD TEMPERATURE
✓ Temp loaded: 130 days.
  Range: 26.07–32.72 °C

LOAD CURVE6 & CURVE7 (Rhodo25_data)
✓ Curve6 loaded (78 increments)
✓ Curve7 loaded (81 increments)

STEP 3: SCREENING (Linear + Curve6/7 for AFE5-1)
  afe5-1: Linear R² = 0.4106
      curve6 time R² = 0.4899
      curve7 time R² = 0.4771
  afe5-2: Linear R² = 0.5011
  afe5-3: Linear R² = 0.4610
  afe5-4: Linear R² = 0.6092
  afe5-5: Linear R² = 0.0707
  afe5-6: Linear R² = 0.0764
  afe5-7: Linear R² = 0.3628

✓ WINNER: afe5-4 using linear time axis
  Mg/Sr = 2.0577 * Temp + 13.4355
  Best R² = 0.6092

STEP 4: GENERATE SYNTHETIC Mg/Sr MASTER
✓ Synthetic Master Created (130 days)
  Mg/Sr range: 67.08–80.75 mmol/mol

STEP 5: UNIFIED DTW ALIGNMENT (window=30 days)
  ✓ Aligned afe5-1: 156 steps, stretch=1.20
  ✓ Aligned afe5-2: 147 steps, stretch=1.

 ## Block bootstrap (Mg/Sr)



 Resamples the calibration data in contiguous blocks to account for

 temporal autocorrelation. Returns 95 % CIs for slope, intercept, and RMSE.

In [2]:
import pandas as pd

boot_results = pipeline.block_bootstrap_calibration(
    proxy='Mg/Sr',
    block_len=7,
    n_boot=1000,
)

if boot_results:
    ci_df = pd.DataFrame({
        'Parameter': ['slope', 'intercept', 'RMSE (°C)'],
        '2.5 %':     [boot_results['slope_ci'][0],
                      boot_results['intercept_ci'][0],
                      boot_results['rmse_ci'][0]],
        'Median':    [boot_results['slope_ci'][1],
                      boot_results['intercept_ci'][1],
                      boot_results['rmse_ci'][1]],
        '97.5 %':    [boot_results['slope_ci'][2],
                      boot_results['intercept_ci'][2],
                      boot_results['rmse_ci'][2]],
    })
    print('\nBootstrap 95 % confidence intervals:')
    print(ci_df.to_string(index=False, float_format='%.4f'))


BLOCK BOOTSTRAP (proxy=Mg/Sr, block_len=7, n_boot=1000)
  Slope 95% CI: 0.4441 – 0.5524
  RMSE 95% CI:  0.336 – 0.718
  Effective bootstraps: 1000


Bootstrap 95 % confidence intervals:
Parameter    2.5 %  Median  97.5 %
    slope   0.4441  0.5071  0.5524
intercept -11.5938 -8.2072 -3.5526
RMSE (°C)   0.3357  0.5316  0.7180


 ## Permutation test (Mg/Sr)



 Permutes the logger temperature record while preserving autocorrelation,

 refits the calibration, and reports the proportion of null R² values that

 meet or exceed the observed R².

In [3]:
from rhodopipeline import permutation_test_mgsr

perm_results = permutation_test_mgsr(pipeline, n_perm=500, block_len=7)

print(f'\nObserved R²: {perm_results["observed_r2"]:.3f}')
print(f'Circular-shift    p-value: {(perm_results["shift_r2"] >= perm_results["observed_r2"]).mean():.3f}')
print(f'Block-permutation p-value: {(perm_results["block_r2"] >= perm_results["observed_r2"]).mean():.3f}')


Observed Mg/Sr R² = 0.909 (n=130)

Permutation test (circular shifts):
  Median null R²         = 0.187
  95th percentile null R² = 0.661
  Proportion null R² >= observed = 0.000

Permutation test (block permutations):
  Median null R²         = 0.024
  95th percentile null R² = 0.177
  Proportion null R² >= observed = 0.000

Observed R²: 0.909
Circular-shift    p-value: 0.000
Block-permutation p-value: 0.000


 ## Flat-master null model



 Replaces the temperature-derived Mg/Sr master with a constant (flat) series

 and re-runs the full DTW + calibration pipeline. The resulting R² sets a

 lower-bound baseline for what pure DTW flexibility can achieve without any

 real temperature signal.

In [4]:
from rhodopipeline import proxy_only_null_flat_master

r2_null = proxy_only_null_flat_master(pipeline, window_days=30)

real_r2 = pipeline.final_equations['Mg/Sr']['stats']['r2']
print(f'\nReal Mg/Sr R²       : {real_r2:.3f}')
print(f'Flat-master null R² : {r2_null:.3f}')
print(f'Improvement         : {real_r2 - r2_null:+.3f}')



=== PROXY-ONLY NULL: FLAT MASTER ===
STEP 5: UNIFIED DTW ALIGNMENT (window=30 days)
  ✓ Aligned afe5-1: 130 steps, stretch=1.00
  ✓ Aligned afe5-2: 130 steps, stretch=1.00
  ✓ Aligned afe5-3: 130 steps, stretch=1.00
  ✓ Aligned afe5-4: 130 steps, stretch=1.00
  ✓ Aligned afe5-5: 130 steps, stretch=1.00
  ✓ Aligned afe5-6: 130 steps, stretch=1.00
  ✓ Aligned afe5-7: 130 steps, stretch=1.00

DTW stretch factors:
branch  stretch_factor
afe5-1             1.0
afe5-2             1.0
afe5-3             1.0
afe5-4             1.0
afe5-5             1.0
afe5-6             1.0
afe5-7             1.0

Mean stretch factor: 1.00

STEP 6: COMPOSITE & FINAL CALIBRATION
  Mg/Sr Final Model: Temp = 0.4571*Mg/Sr + -4.6976
    R²=0.310, RMSE=1.742, n=130

  Mg/Ca Final Model: Temp = 0.0096*Mg/Ca + 26.6803
    R²=0.002, RMSE=2.096, n=130

  Sr/Ca Final Model: Temp = -6.5942*Sr/Ca + 55.5669
    R²=0.203, RMSE=1.872, n=130

Flat-master Mg/Sr R² = 0.310

Real Mg/Sr R²       : 0.310
Flat-master null R² : 0.

 ## Multi-element screening per branch

In [5]:
pipeline.screen_all_elements_per_branch()



STEP 11: SCREENING ALL ELEMENTS PER BRANCH (Mg/Sr Aligned)

--- afe5-1 ---
Element    R2       Slope       n
   K/Ca 0.402      -1.176 151.000
  La/Ca 0.339 -585369.125 150.000
  Mo/Ca 0.308  -18467.185 150.500
  Sr/Ca 0.280      -8.813 151.000
  Mg/Ca 0.184       0.047 151.000
  Ce/Ca 0.162 -522455.172 151.000
  Al/Ca 0.103     366.093 121.000
  Na/Ca 0.086      -0.127 151.000
  Li/Ca 0.071      19.866 151.000
  Pb/Ca 0.030   -2442.644 151.000
   B/Ca 0.024       3.725 151.000
  Ba/Ca 0.019     343.430 151.000
  Fe/Ca 0.017     -20.683 140.000
   U/Ca 0.005   26908.669 151.000
  Mn/Ca 0.001     -27.462 146.000

--- afe5-2 ---
Element    R2       Slope       n
  Ba/Ca 0.613   -2943.650 143.000
  Mg/Ca 0.334       0.047 143.000
   B/Ca 0.312      17.276 143.000
  Pb/Ca 0.297    4561.287 143.000
  Al/Ca 0.203    1025.208  86.000
  La/Ca 0.195 -290392.041 142.000
  Mo/Ca 0.179    6947.108 143.000
   U/Ca 0.166 -114019.606 143.000
  Na/Ca 0.139      -0.094 143.000
  Sr/Ca 0.095      -3.76